# Summary
This notebook takes objects*(kodeoversikt*.csv) from nin3 code api and checks if they get a match in cms api (the adb-api in front of drupal cms)

(* : Type, Hovedtypegruppe, Hovedtype, Grunntype etc.)

# Load kodeoversikt.csv to a dataframe

In [ ]:
import pandas as pd

# Read the CSV file into a dataframe
def read_csv(file):
    df = pd.read_csv(file, sep=";")
    return df
df = read_csv("kodeoversikt_chunk.csv") # kodeoversikt_chunk.csv: minimal subset of kodeoversikt.csv. (5 of each object type)
                                        # ..Kodeoversikt.csv can be used to scan cms for all codes for nin3. (slow)
df.info()
# Display the dataframe

# Create sample csv, a chunk of original

In [ ]:
new_dfs = []
for klasse in df['Klasse'].unique():
    new_df = df[df['Klasse'] == klasse].head(5)
    new_dfs.append(new_df)

# Concatenate all the new dataframes into a single dataframe
new_df = pd.concat(new_dfs)

# Write the new dataframe to a CSV file
new_df.to_csv('kodeoversikt_chunk.csv', index=False, sep=";")


In [ ]:
def lookup_code(code):
    import requests
    url_code_lookup_base = "https://artsdatabanken.no/api/nin/v3?code="
    url = url_code_lookup_base + code
    return requests.get(url)


## for each row in DataFrame - query cms-api with ['Langkode']
* uses current dataframe with nin3 langkode's to query cms-api and fetches http-status(200OK, 404NOTFound) and part of json-content
* output: csv report with results that tell if a langkode exist in cms-api and if it has data.


In [ ]:
from IPython.display import clear_output
progress_intervall = 5
df["status_code"] = ""
df["in_drupal"] = ""
df["content_preview"] = ""
for index, row in df.iterrows():
    code = row['Langkode']
    response = lookup_code(code)
    if response.status_code == 200:
        df.at[index, 'in_drupal'] = 'True'
        df.at[index, 'content_preview'] = response.json()[:50]
    else:
        df.at[index, 'in_drupal'] = 'False'
    df.at[index, 'status_code'] = response.status_code
    if index % progress_intervall == 0:
        clear_output(wait=True)
        display(f"Number of rows iterated: {index+1}")
        #display(df)
df.to_csv('output_kodeoversikt.csv', index=False, sep=";")
print("Done! Look at output_kodeoversikt.csv")